In [1]:
# importing libraries 
import pandas as pd
import torch
from tqdm import tqdm
from local_transformers.src.transformers import TrainingArguments
from local_transformers.src.transformers.models.biogpt.tokenization_biogpt import BioGptTokenizer # reads the local_transformers folder instead of the actual transformers package
from local_transformers.src.transformers.models.CustomBioGPT import CustomBioGptForCausalLM # reads the local_transformers folder instead of the actual transformers package
from local_transformers.src.transformers.CustomComputeLoss import CustomTrainer # reads the local_transformers folder instead of the actual transformers package
import torch.nn.functional as F
import pickle
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
import xgboost as xgb
import numpy as np
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from torch.utils.data import ConcatDataset
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available, running on CPU')

# Reading data
df=pd.read_csv("df.csv")
df["death_in_30"] = df["death_in_30"].astype(int)
df["DVT"] = df["DVT"].astype(int)
df["PE"] = df["PE"].astype(int)
df["post_aki_status"] = df["post_aki_status"].astype(int)
df["PNA"] = df["PNA"].astype(int)


# VERY IMPORTANT: MAKE SURE TO USE CustomBioGptForCausalLM SO WE CAN ACCEPT THE LABELS AS PART OF OUR FINE-TUNNING 
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")

# preparing the clinical notes for training and fine-tuning 
# this function here helps set up and prepare the data for fine-tuning, including those of the additional labels. 
class preparing(torch.utils.data.Dataset):
    def __init__(self, notes, additional_labels, tokenizer, task_ids, max_length=188):
        self.notes = notes
        self.additional_labels = additional_labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.task_ids=task_ids

    def __getitem__(self, idx):
        note = self.notes[idx]
        additional_label = self.additional_labels[idx]
        task_id = self.task_ids[idx]

        # Tokenize the input note
        encoding = self.tokenizer(note, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        # Split the input sequence into input and label sequences
        input_ids = encoding['input_ids'][0][:-1]
        label_ids = encoding['input_ids'][0][1:]

        # Return the input and label sequences along with the additional_label
        return {
            'input_ids': input_ids,
            'attention_mask': encoding['attention_mask'][0][:-1],
            'labels': label_ids,
            'additional_labels': additional_label, # HERE WE INCLUDE THE ADDITIONAL_LABELS TO BE CONSIDERED AS PART OF OUR TRAINING
            'task_ids': task_id # Include the task_ids to differentiate tasks
            }

    def __len__(self):
        return len(self.notes)


# By default Trainer actually shuffles the order of the data, but we can use this as well if we wish to
# class ShuffledDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset):
#         self.dataset = dataset
#         indices = torch.randperm(len(dataset))
#         self.shuffled_data = [dataset[i] for i in indices]

#     def __getitem__(self, idx):
#         return self.shuffled_data[idx]

#     def __len__(self):
#         return len(self.dataset)

# stacking the data of different labels on top of one another so we can perform MTL fine-tuning
def stack_data(df,tasks):
    tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
    datasets = []
    # print(enumerate(tasks))
    for i, task in enumerate(tasks):
        df_now = df.dropna(subset=[task]).reset_index(drop=True)
        dataset = preparing(df_now["AN_PROC_NAME"], df_now[task], tokenizer, torch.tensor([i] * len(df_now)))
        datasets.append(dataset)

    combined_dataset = ConcatDataset(datasets)
    return(combined_dataset)

/home/charles/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-11-04 03:27:44.133306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-04 03:27:44.602055: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [2]:
len(df)

84875

In [2]:
# this finetunes our model
def get_model(name, train_dataset,val_dataset):
    model = CustomBioGptForCausalLM.from_pretrained("microsoft/biogpt", output_hidden_states=True, lambda_constant=10, num_tasks=6)
    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=3,      # total number of training epochs
        per_device_train_batch_size=48,  # batch size per device during training
        per_device_eval_batch_size=48,   # batch size for evaluation
        warmup_steps=1000,          # number of warmup steps for learning rate scheduler
        weight_decay=0.001,          # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=1000,  # Add this line
        save_strategy="steps",
        learning_rate=0.000005)       
    # Create the Trainer object
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        lambda_constant=10)

    # Fine-tune BioGPT on the training dataset
    trainer.train()
    #trainer.evaluate(eval_dataset=val_dataset)
    # saving the newly pre-trained model. 
    trainer.model.save_pretrained(name)

In [3]:
# gets the text representation from the updated specified model
def get_biogpt_updated(texts, name):
    batch_size=64
    max_length=144
    texts = list(texts)

    model = CustomBioGptForCausalLM.from_pretrained(name, output_hidden_states=True, lambda_constant=10, num_tasks=6) 
    tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")


    # Move the model to the GPU if available
    if torch.cuda.is_available():
        model = model.to("cuda")
    # tokenize the input text in batches
    all_embeddings = []
    for i in (range(0, len(texts), batch_size)):
        print("Progress", (i/len(texts))*100)
        batch_texts = texts[i:i + batch_size]
        batch_encoded = tokenizer(batch_texts, padding='max_length', truncation=True, return_tensors="pt",
                                  max_length=max_length)
        input_ids = batch_encoded["input_ids"]
        attention_mask = batch_encoded["attention_mask"]

        # Move the tensors to the GPU if available
        if torch.cuda.is_available():
            input_ids = input_ids.to("cuda")
            attention_mask = attention_mask.to("cuda")

        # process each batch separately
        batch_embeddings = []
        for j in (range(input_ids.shape[0])):
            with torch.no_grad():
                outputs = model(input_ids[j:j + 1], attention_mask=attention_mask[j:j + 1])
                last_hidden_state = outputs.hidden_states[-1]
                masked_hidden_state = last_hidden_state * attention_mask[j:j + 1].unsqueeze(-1)
                embedding = torch.mean(masked_hidden_state, dim=1)
                embedding = F.normalize(embedding, p=2, dim=1)
            
            # If you used GPU, move the embeddings back to CPU for further operations (like converting to numpy array)
            if torch.cuda.is_available():
                embedding = embedding.to("cpu")
            batch_embeddings.append(embedding)

        # concatenate the embeddings from all examples in the batch
        embeddings = torch.cat(batch_embeddings, dim=0)
        all_embeddings.append(embeddings)

    # concatenate the embeddings from all batches
    embeddings = torch.cat(all_embeddings, dim=0)

    # convert the embeddings to a numpy array
    X = embeddings.detach().numpy()

    return (X)

In [4]:
# performs the 5-fold for XGBoost (default model)
def rounding(take_mean, take_std):
    return(round(take_mean, 2), round(take_std,2))


def K_fold_val(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    # Initialize lists to hold scores
    auroc_scores, auprc_scores,accuracy_scores, recall_scores,precision_scores, specificity_scores,f_scores = [],[],[],[],[],[],[]
    # Define the parameter grid
    param_grid = {
        'learning_rate': [0.1,0.15,0.3],
        'max_depth': [4,5,6,7,8],
        'min_child_weight':[1,2,4]
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        # Split the data into train/test sets
        train, test = df.iloc[train_index], df.iloc[test_index]
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            # train val split
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_notes=(train_new.reset_index(drop=True))
            val_notes=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_dataset=stack_data(train_notes,tasks)
            val_dataset=stack_data(val_notes,tasks)
            get_model(name, train_dataset,val_dataset)
        if os.path.isfile(f'{name}.pickle'):
            with open(f'{name}.pickle', 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_biogpt_updated(train["AN_PROC_NAME"], name)
            X_test = get_biogpt_updated(test["AN_PROC_NAME"], name)
            with open(f"{name}.pickle", 'wb') as f:
                    pickle.dump([X_train,X_test], f)  

        # Prepare training and testing data
        y_train = train[outcome_col]
        y_test = test[outcome_col]

         # removing rows with potential NAs in them
        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        
    # performing the XGboost classification
        model = xgb.XGBClassifier(random_state=42,tree_method='gpu_hist')
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]
        y_pred_class = best_model.predict(X_test)

        # Compute the metrics
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)
        accuracy=accuracy_score(y_test, y_pred_class)
        recall = recall_score(y_test, y_pred_class)
        precision = precision_score(y_test, y_pred_class)
        # For specificity (True Negative Rate)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()
        specificity = tn / (tn + fp)
        f_score=f1_score(y_test, y_pred_class)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        accuracy_scores.append(accuracy)
        recall_scores.append(recall)
        precision_scores.append(precision)
        specificity_scores.append(specificity)
        f_scores.append(f_score)

        with open(f"update_{outcome_col}.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores,"accuracy":accuracy_scores,
                         "precision":precision_scores,"recall":recall_scores, 
                         "specificity":specificity_scores,
                         "f1":f_scores},f)

    
    # Now you can calculate the mean and standard deviation
    mean_auroc,std_auroc = rounding(np.mean(auroc_scores), np.std(auroc_scores))
    mean_auprc, std_auprc = rounding(np.mean(auprc_scores), np.std(auprc_scores))
    mean_accuracy, std_accuracy=rounding(np.mean(accuracy_scores), np.std(accuracy_scores))
    mean_precision, std_precision=rounding(np.mean(precision_scores), np.std(precision_scores))
    mean_recall, std_recall=rounding(np.mean(recall_scores), np.std(recall_scores))
    mean_specificity, std_specificity=rounding(np.mean(specificity_scores), np.std(specificity_scores))
    mean_f1,std_f1=rounding(np.mean(f_scores), np.std(f_scores))  
    
    print_statement=f'''
    metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, 
    \n Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}, 
    \n Mean ACC:{mean_accuracy}, SD ACC:{std_accuracy}, 
    \n Mean precision:{mean_precision}, std precision:{std_precision}, 
    \n mean recall:{mean_recall}, std_recall:{std_recall},
    \n mean specificity:{mean_specificity}, std specificity:{std_specificity}, 
    \n mean f1:{mean_f1}, std f1:{std_f1} 
    '''
    return(print_statement)


In [5]:
# performs the 5-fold for Logistic Regression
def K_fold_val_log_reg(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []

    # changed parameters here because initial parameters gave underwhelming results
    # so i explored alternative hyperparameters to see if model improves
    param_grid = {
    'C': [0.01, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['lbfgs', 'newton-cholesky']
    }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"            
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_new=(train_new.reset_index(drop=True))
            val_new=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_new=stack_data(train_new, tasks)
            val_new=stack_data(val_new, tasks)
            get_model(name, train_new,val_new)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_biogpt_updated(train["AN_PROC_NAME"], name)
            X_test = get_biogpt_updated(test["AN_PROC_NAME"], name)
                    
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)
        # Prepare training and testing data

        y_train = train[outcome_col]
        y_test = test[outcome_col]

        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        
        model = LogisticRegression(random_state=42,max_iter=10000)
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}_logreg.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


In [6]:
# # performs the 5-fold for Random Forest
def K_fold_val_rf(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []

    # changed parameters here because initial parameters gave underwhelming results
    # so i explored alternative hyperparameters to see if model improves
    param_grid = {
        'max_depth': [4, None],  # maximum depth of the tree
        'min_samples_split': [2, 5],  # minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 3],  # minimum number of samples required to be at a leaf node
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"            
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_new=(train_new.reset_index(drop=True))
            val_new=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_new=stack_data(train_new, tasks)
            val_new=stack_data(val_new, tasks)
            get_model(name, train_new,val_new)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_biogpt_updated(train["AN_PROC_NAME"], name)
            X_test = get_biogpt_updated(test["AN_PROC_NAME"], name)
                    
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)
        # Prepare training and testing data

        y_train = train[outcome_col]
        y_test = test[outcome_col]

        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        
        model = RandomForestClassifier(random_state=42)
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}_rf.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


In [5]:
results=K_fold_val("death_in_30", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

1it [05:29, 329.73s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

2it [11:01, 330.73s/it]

...

3it [16:29, 329.53s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

4it [21:58, 329.52s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

5it [27:28, 329.71s/it]


In [8]:
results=K_fold_val_log_reg("death_in_30", df)

0it [00:00, ?it/s]

passed
Progress 0.0
Progress 0.09425625920471281
Progress 0.18851251840942562
Progress 0.28276877761413843
Progress 0.37702503681885124
Progress 0.47128129602356406
Progress 0.5655375552282769
Progress 0.6597938144329897
Progress 0.7540500736377025
Progress 0.8483063328424153
Progress 0.9425625920471281
Progress 1.036818851251841
Progress 1.1310751104565537
Progress 1.2253313696612664
Progress 1.3195876288659794
Progress 1.4138438880706923
Progress 1.508100147275405
Progress 1.602356406480118
Progress 1.6966126656848306
Progress 1.7908689248895433
Progress 1.8851251840942562
Progress 1.979381443298969
Progress 2.073637702503682
Progress 2.167893961708395
Progress 2.2621502209131075
Progress 2.3564064801178204
Progress 2.450662739322533
Progress 2.544918998527246
Progress 2.6391752577319587
Progress 2.7334315169366716
Progress 2.8276877761413846
Progress 2.921944035346097
Progress 3.01620029455081
Progress 3.110456553755523
Progress 3.204712812960236
Progress 3.2989690721649487
Progress

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06596e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.4s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06167e-08): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12779e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.4s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12196e-08): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


5it [1:38:49, 1185.84s/it]


In [10]:
results=K_fold_val_rf("death_in_30", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.1s
[CV] END max_depth=4, min_sampl

1it [1:36:40, 5800.42s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_sampl

2it [3:05:43, 5531.21s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.1s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.2s
[CV] END max_depth=4, min_sampl

3it [4:45:24, 5736.80s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_sampl

4it [6:17:25, 5651.37s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.3s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.4s
[CV] END max_depth=4, min_sampl

## Part II: DVT

In [7]:
results=K_fold_val("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [04:59, 299.13s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [09:55, 297.25s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [14:51, 296.68s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
4it [19:46, 296.15s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [24:45, 297.12s/it]


In [7]:
results=K_fold_val_log_reg("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.6s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06581e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.4s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06222e-08): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12942e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.4s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12147e-08): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


5it [04:53, 58.64s/it]


In [8]:
results=K_fold_val_rf("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.7s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.6s
[CV] END max_depth=4, min_sampl

1it [1:34:54, 5694.17s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.8s
[CV] END max_depth=4, min_sampl

2it [3:18:57, 6017.44s/it]

...

3it [5:06:43, 6222.24s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.8s
[CV] END max_depth=4, min_sampl

4it [6:46:13, 6122.56s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.7s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_sampl

5it [8:42:07, 6265.42s/it]


## Part III: PE

In [9]:
results=K_fold_val("PE", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [04:48, 288.29s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [09:36, 287.95s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [14:22, 287.19s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4

4it [19:09, 287.15s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [23:57, 287.55s/it]


In [10]:
results=K_fold_val_log_reg("PE", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.6s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06579e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.4s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06207e-08): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12915e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.4s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12169e-08): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.4s


5it [04:48, 57.79s/it]


In [12]:
results=K_fold_val_rf("PE", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_sampl

1it [1:38:01, 5881.01s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_sampl

2it [3:08:05, 5600.69s/it]

...

3it [4:45:00, 5698.46s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.8s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_sampl

4it [6:23:08, 5773.41s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_sampl

5it [7:47:31, 5610.38s/it]


## Part 4: PNA

In [11]:
results=K_fold_val("PNA", df)


0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [04:57, 297.43s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [09:55, 297.54s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [14:52, 297.41s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
4it [19:50, 297.60s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [24:47, 297.47s/it]


In [14]:
results=K_fold_val_log_reg("PNA", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.6s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06579e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06206e-08): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12937e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12162e-08): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


5it [04:17, 51.58s/it]


In [8]:
results=K_fold_val_rf("PNA", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.6s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_sampl

1it [1:39:31, 5971.26s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_sampl

2it [3:17:07, 5903.59s/it]

...

3it [4:59:34, 6014.87s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.1s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_sampl

4it [6:35:03, 5901.78s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  37.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  38.0s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=4, min_sampl

5it [8:19:47, 5997.53s/it]


## Part 5: AKI

In [13]:
results=K_fold_val("post_aki_status", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

1it [06:10, 370.78s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

2it [12:18, 369.21s/it]

...

3it [18:28, 369.19s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

4it [24:35, 368.41s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4

5it [30:42, 368.59s/it]


In [10]:
results=K_fold_val_log_reg("post_aki_status", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.8s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06576e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.06193e-08): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12886e-08): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.12147e-08): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


5it [09:01, 108.37s/it]


In [12]:
results=K_fold_val_rf("post_aki_status", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  35.9s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  35.9s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  36.0s
[CV] END max_depth=4, min_sampl

1it [1:10:01, 4201.91s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  36.3s
[CV] END max_depth=4, min_sampl

2it [2:21:49, 4263.95s/it]

...

3it [3:35:05, 4324.12s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.4s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  36.4s
[CV] END max_depth=4, min_sampl

4it [4:48:46, 4362.76s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=  36.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.3s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.2s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=  36.2s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=  36.2s
[CV] END max_depth=4, min_sampl

5it [6:01:29, 4337.87s/it]


## Part 6: delirium

In [15]:
results=K_fold_val("postop_del", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4

1it [03:02, 182.76s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4

2it [06:05, 182.89s/it]

...

3it [09:07, 182.48s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4

4it [12:10, 182.55s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4

5it [15:10, 182.20s/it]


In [14]:
results=K_fold_val_log_reg("postop_del", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
   

In [16]:
results=K_fold_val_rf("postop_del", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_sampl

1it [05:43, 343.53s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_sampl

2it [11:29, 345.04s/it]

...

3it [17:18, 346.96s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_sampl

4it [23:04, 346.34s/it]

passed
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=5; total time=   4.5s
[CV] END max_depth=4, min_samples_leaf=3, min_samples_split=2; total time=   4.5s
[CV] END max_depth=4, min_sampl

5it [28:48, 345.78s/it]
